## Question

How to find and inspect a Metaflow task with a certain artifact value? This question comes up often in the context of [foreaches](https://docs.metaflow.org/metaflow/basics#foreach) that yield many tasks from a single step. For instance, you might train a separate model for each country and afterwards you want to inspect data related to a specific country.

## Answer

You can use the [Metaflow Client API](https://docs.metaflow.org/metaflow/client) with Python's [list comprehensions](https://realpython.com/list-comprehension-python/) to find tasks that have desired data. A benefit of this approach is that you can use familiar Python constructs, such as `and` and `or`, to express arbitrarily complex queries over data.

## Example

Here's a flow with a foreach branch that yields three tasks for _South Korea_, _Sweden_, and _Cuba_:

In [1]:
%%writefile finding_tasks_country_flow.py
from metaflow import FlowSpec, step

class CountryFlow(FlowSpec):

    @step
    def start(self):
        self.countries = ['South Korea', 'Sweden', 'Cuba']
        self.next(self.process_country, foreach='countries')
        
    @step
    def process_country(self):
        self.country = self.input
        self.length = len(self.input)
        self.next(self.join)
        
    @step
    def join(self, inputs):
        self.next(self.end)

    @step
    def end(self):
        pass

if __name__ == '__main__':
    CountryFlow()

Overwriting finding_tasks_country_flow.py


Run the flow:

In [2]:
!python finding_tasks_country_flow.py run

Metaflow 2.5.2 executing CountryFlow for user:ville
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2022-02-22 14:15:48.227 Workflow starting (run-id 1645568148219212):
2022-02-22 14:15:48.233 [1645568148219212/start/1 (pid 33726)] Task is starting.
2022-02-22 14:15:48.772 [1645568148219212/start/1 (pid 33726)] Foreach yields 3 child steps.
2022-02-22 14:15:48.772 [1645568148219212/start/1 (pid 33726)] Task finished successfully.
2022-02-22 14:15:48.781 [1645568148219212/process_country/2 (pid 33729)] Task is starting.
2022-02-22 14:15:48.788 [1645568148219212/process_country/3 (pid 33730)] Task is starting.
2022-02-22 14:15:48.794 [1645568148219212/process_country/4 (pid 33731)] Task is starting.
2022-02-22 14:15:49.341 [1645568148219212/process_country/3 (pid 33730)] Task finished successfully.
2022-02-22 14:15:49.343 [1645568148219212/process_country/2 (pid 33729)] Task finished successfully.
2022-02-22 14:15:49.353 [1645568148219212/process_

Next, let's use the Client API to inspect the run we just executed. You can execute the following commands e.g. in a notebook.

In [3]:
from metaflow import Flow
run = Flow('CountryFlow').latest_run
run

Run('CountryFlow/1645568148219212')

You can see that the run ID matches with the output of our run above. Now that we have a `Run` object that corresponds to the run we want to inspect, we can use a list comprehension to find tasks with desired data.

Here's how to find a task corresponding to _Cuba_:

In [4]:
[cuba] = [task for task in run['process_country']
          if task['country'].data == 'Cuba']
cuba

Task('CountryFlow/1645568148219212/process_country/4')

Now that we have found a `Task` object corresponding to _cuba_, we can query any data produced by the task:

In [5]:
cuba.data.length

4

You can use any Python expressions inside list comprehensions to express complex queries. This example finds all tasks where the country name includes the letter _e_ and the name length is longer than 3:

In [6]:
[task['country'].data for task in run['process_country']
 if 'e' in task['country'].data and
     task['length'].data > 3]

['Sweden', 'South Korea']

## See Also|
 - [Finding Tasks Based on Tags](finding-tasks-based-on-artifact-values)